In [1]:
import numpy as np
import random
import time
from IPython.display import clear_output

class TownBoard:
    def __init__(self, size=20, num_houses=10):
        self.size = size
        self.num_houses = num_houses
        self.grid = np.zeros((size, size), dtype=int)
        self.cell_types = {
            0: {'name': 'tree', 'emoji': '🌲', 'flammable': True},
            1: {'name': 'house', 'emoji': '🏠', 'flammable': True},
            2: {'name': 'fire', 'emoji': '🔥', 'flammable': False},
            3: {'name': 'burned', 'emoji': '🪵', 'flammable': False},
            4: {'name': 'road', 'emoji': '⬜', 'flammable': False},
            5: {'name': 'tower', 'emoji': '🗼', 'flammable': False},
            6: {'name': 'car', 'emoji': '🚗', 'flammable': False},
            13: {'name':'tornado','emoji':'🌪️','flammable':False},
            14: {'name':'destroyed', 'emoji': '🪵', 'flammable': False}
        }
        self.warning_active = False
        self.house_paths = []
        self.cars = []
        self.car_times = []

    def generate_town(self):
        self.grid.fill(0)
        self._create_crossroads()  # Add main roads
        self.grid[1][1] = 5
        self.house_positions = []
        placed = 0
        while placed < self.num_houses:
            x, y = random.randint(1, self.size - 2), random.randint(1, self.size - 2)
            if self.grid[y][x] == 0:
                self.grid[y][x] = 1
                path = self._connect_to_cross(x, y)  # Connect house to road
                if path:
                    self.house_positions.append((x, y))
                    self.house_paths.append(path)
                    placed += 1

    def _create_crossroads(self):
        # Create T-shaped crossroad in center
        mid = self.size // 2
        for i in range(self.size):
            self.grid[mid][i] = 4  # Horizontal
            self.grid[i][mid] = 4  # Vertical
    
    def _connect_to_cross(self, x, y):
        # Create road path from a house at (x,y) to the nearest crossroad
        mid = self.size // 2
        path = []
        cx, cy = x, y

        # Move horizontally to center
        while cx != mid:
            cx += 1 if mid > cx else -1
            if self.grid[cy][cx] == 0:
                self.grid[cy][cx] = 4
            path.append((cx, cy))

        # Move vertically to center
        while cy != mid:
            cy += 1 if mid > cy else -1
            if self.grid[cy][cx] == 0:
                self.grid[cy][cx] = 4
            path.append((cx, cy))

        # From center, randomly pick an edge as the road exit
        exits = [(0, cy), (self.size - 1, cy), (cx, 0), (cx, self.size - 1)]
        ex, ey = random.choice(exits)

        # Extend road horizontally to edge
        while cx != ex:
            cx += 1 if ex > cx else -1
            if self.grid[cy][cx] == 0:
                self.grid[cy][cx] = 4
            path.append((cx, cy))

        # Extend road vertically to edge
        while cy != ey:
            cy += 1 if ey > cy else -1
            if self.grid[cy][cx] == 0:
                self.grid[cy][cx] = 4
            path.append((cx, cy))

        return path

    def assign_cars(self):
        for path in self.house_paths:
            if not path:
                continue
            start = path[0]
            x, y = start
            delay = 0
            self.cars.append([x, y, path[:], delay, 0])  # (x, y, path, delay, steps)

    def move_cars(self):
        updated_cars = []
        for x, y, path, delay, steps in self.cars:
            if delay > 0:
                updated_cars.append([x, y, path, delay - 1, steps + 1])
                continue
            self.grid[y][x] = 4  # Leave behind a road
            if path:
                x, y = path.pop(0)
                if 0 <= x < self.size and 0 <= y < self.size:
                    self.grid[y][x] = 6
                    updated_cars.append([x, y, path, 0, steps + 1])
            else:
                self.car_times.append(steps)
        self.cars = updated_cars

    def display(self):
        clear_output(wait=True)
        if self.warning_active:
            print("\n🚨 TORNADO WARNING 🚨\n🚨 WEE-OO WEE-OO 🚨\n")
        for row in self.grid:
            print(' '.join(self.cell_types[cell]['emoji'] for cell in row))
        print("\nLegend: 🌲 Tree | 🏠 House | 🌪️ Tornado | 🪵 Destroyed Area | ⬜ Road | 🗼 Tower | 🚗 Car")

In [50]:
class TornadoSimulator:
    """
    Simulates a tornado moving through the town
    
    Attributes:
    ----------
    board : TownBoard class the simulation is over
    time_step : int, number of steps passed
    path : list of the coordinates the tornado has moved through
    under : dictionary of cells the tornado passed over
    destroyed_cars : list of cars destroyed by the tornado

    Methods:
    ----------
    __init__(town_board):
        Initializes the simulator
        Includes starting the tornado on a random edge cell
        and starting the warning for evacuation.

    move():
        Moves the tornado randomly, destroys non-road cells
        under the tornado, and checks if the tornado hits a car.
        
    hit_check():
        Checks whether the tornado is on any cars and destroys those cars
    
    simulate(steps,delay):
        Run complete tornado simulation by displaying the board and
        moving each time step
    """
    def __init__(self, town_board):
        """
        Initializes the simulator
        Includes starting the tornado on a random edge cell
        and starting the warning for evacuation.
        
        Paramaters:
        town_board : TownBoard class the simulation is on
        """
        #sets initial attributes
        self.board = town_board
        self.time_step = 0
        self.destroyed_cars=[]

        #randomly picks coordinates among the cells on the edges of the board
        x, y = random.randint(0, len(self.board.grid)-1), random.randint(0, len(self.board.grid)-1)
        starts=[(0,y),(x,0),(len(self.board.grid)-1,y),(x,len(self.board.grid)-1)]
        start=starts[np.random.choice(len(starts))]
        #records what's already there and sets the tornado
        self.under={start:self.board.grid[start]}
        self.board.grid[start]=13
        #adds the coordinates to the path
        self.path = [start]

        #starts evacuation
        self.board.warning_active = True
        self.board.assign_cars()
        
    
    def move(self):
        """
        Moves the tornado randomly, destroys non-road cells
        under the tornado, and checks if the tornado hits a car.
        """
        #gets last tornado coordinates
        x,y=self.path[-1]

        #sets what's under the tornado to destroyed or back to a road
        previous=self.under[(x,y)]
        if previous!=4:
            self.board.grid[(x,y)]=14
        else:
            self.board.grid[(x,y)]=4

        #Makes a weighted list of spaces in a 3x3 area around the tornado where
        #it could move, discouraging looping back on itself or staying still
        options=[]
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                nx, ny = x+dx, y+dy
                if dx == 0 and dy == 0:
                    for k in range(2):
                        options.append((nx,ny))
                if 0<=nx<len(self.board.grid) and 0<=ny<len(self.board.grid):
                    if (nx,ny) in self.path:
                        options.append((nx,ny))
                    else:
                        for k in range(3):
                            options.append((nx,ny))  
        
        #Randomly picks the cell to move to
        new=options[np.random.choice(len(options))]
        
        #Adds what's already there before the tornado moves on
        if self.board.grid[new]==6:
            self.under[new]=4 #cars are ignored and interpreted as roads
        else:
            self.under[new]=self.board.grid[new]

        #sets the cell to the tornado and adds it to the path
        self.board.grid[new]=13
        self.path.append(new)
        #checks whether the tornado is on any cars
        self.hit_check()
        self.time_step += 1

        
    def hit_check(self):
        """
        Checks whether the tornado is on any cars and destroys those cars
        """
        cars=[]
        for car in self.board.cars:
            #checks if each car's coordinates are the same as the tornado's
            if car[0]==self.path[-1][0] and car[1]==self.path[-1][1]:
                #records any destroyed cars and sets their cell to road (just in case)
                self.destroyed_cars.append(car)
                self.board.grid[(car[0],car[1])]=4
            else:
                cars.append(car)
                #cars that aren't destroyed are recorded and set
        self.board.cars=cars
                
    
    def simulate(self, steps=25, delay=0.1):
        """
        Run complete tornado simulation by displaying the board and
        moving each time step

        Parameters:
        steps : int, number of steps the simulation loops over
        delay : float, time (in sec) each step is displayed for
        """
        for t in range(steps):
            #each step, displays the board, moves the car, then tornado
            self.board.display()
            self.board.move_cars()
            self.move()
            #stops on the step for the delay length
            time.sleep(delay)

In [49]:
#makes a 15x15 town
town = TownBoard(15)
town.generate_town()
    
# Simulate tornado going through the town
disaster = TornadoSimulator(town)
disaster.simulate(steps=20, delay=0.4)
print("\nSimulation complete")
#shows how many and when each car made it off the board
for i, t in enumerate(town.car_times):
    print(f"Car {i+1} evacuated in {t} minutes")
if town.car_times:
    print(f"Total evacuation time: {max(town.car_times)} minutes")
#shows how many cars were destroyed by the tornado
print(len(disaster.destroyed_cars),'car(s) destroyed')


🚨 TORNADO WARNING 🚨
🚨 WEE-OO WEE-OO 🚨

🌲 🌲 🌲 🌲 🌲 🌲 🌲 ⬜ 🌲 🌲 🌲 🌲 🌲 🌲 🌲
🌲 🗼 🌲 🌲 🌲 🌲 🌲 ⬜ 🏠 🌲 🌲 🌲 🌲 🌲 🌲
🌲 🌲 🌲 🏠 ⬜ ⬜ ⬜ ⬜ 🌲 🌲 🌲 🌲 🌲 🌲 🌲
🌲 🌲 🌲 🌲 🌲 🏠 ⬜ ⬜ 🌲 🌲 🌲 🌲 🌲 🌲 🌲
🌲 🏠 ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ 🌲 🌲 🌲 🌲 🌲 🌲 🌲
🌲 🌲 🌲 🌲 🌲 🏠 ⬜ ⬜ 🌲 🌲 🌲 🌲 🌲 🌲 🌲
🌲 🌲 🌲 🌲 🌲 🏠 ⬜ ⬜ 🌲 🌲 🌲 🌲 🌲 🌲 🌲
⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜ ⬜
🌲 🌲 🌲 🌲 🌲 🌲 🌲 ⬜ 🌲 🌲 🌲 🌲 🌲 🌲 🌲
🌲 🌲 🌲 🌲 🌲 🌲 🌲 ⬜ ⬜ ⬜ ⬜ 🏠 🌲 🌲 🌲
🌲 🌲 🌲 🌲 🌲 🌲 🌲 ⬜ ⬜ ⬜ ⬜ 🏠 🌲 🌲 🌲
🌲 🌲 🌲 🌲 🏠 ⬜ ⬜ ⬜ 🌲 🌲 🌲 🌲 🌲 🌲 🌲
🌲 🌲 🪵 🌲 🌲 🌲 🌲 🌪️ 🌲 🌲 🌲 🌲 🌲 🌲 🌲
🌲 🪵 🪵 🌲 🌲 🌲 🪵 ⬜ 🌲 🌲 🌲 🌲 🌲 🌲 🌲
🪵 🌲 🪵 🪵 🪵 🪵 🪵 ⬜ 🌲 🌲 🌲 🌲 🌲 🌲 🌲

Legend: 🌲 Tree | 🏠 House | 🌪️ Tornado | 🪵 Destroyed Area | ⬜ Road | 🗼 Tower | 🚗 Car

Simulation complete
Car 1 evacuated in 10 minutes
Car 2 evacuated in 11 minutes
Car 3 evacuated in 13 minutes
Car 4 evacuated in 13 minutes
Car 5 evacuated in 13 minutes
Car 6 evacuated in 14 minutes
Car 7 evacuated in 14 minutes
Car 8 evacuated in 14 minutes
Car 9 evacuated in 16 minutes
Car 10 evacuated in 16 minutes
Total evacuation time: 16 minutes
0 car(s) destroyed
